In [84]:
!pip install -r requirements.txt

  Using cached flake8-3.7.9-py2.py3-none-any.whl (69 kB)
  Using cached entrypoints-0.3-py2.py3-none-any.whl (11 kB)
  Using cached pyflakes-2.1.1-py2.py3-none-any.whl (59 kB)
  Using cached pycodestyle-2.5.0-py2.py3-none-any.whl (51 kB)
  Using cached mccabe-0.6.1-py2.py3-none-any.whl (8.6 kB)
  Attempting uninstall: mccabe
    Found existing installation: mccabe 0.7.0
    Uninstalling mccabe-0.7.0:
      Successfully uninstalled mccabe-0.7.0
  Attempting uninstall: pyflakes
    Found existing installation: pyflakes 2.4.0
    Uninstalling pyflakes-2.4.0:
      Successfully uninstalled pyflakes-2.4.0
  Attempting uninstall: pycodestyle
    Found existing installation: pycodestyle 2.8.0
    Uninstalling pycodestyle-2.8.0:
      Successfully uninstalled pycodestyle-2.8.0
  Attempting uninstall: entrypoints
    Found existing installation: entrypoints 0.4
    Uninstalling entrypoints-0.4:
      Successfully uninstalled entrypoints-0.4
  Attempting uninstall: flake8
    Found existing inst

In [85]:
import pandas as pd
# Training Data
# training_df = pd.read_csv(r"C:\Users\ebrady\Personal Projects\tweet-sentiment\data\twitter_training.csv")
training_df = pd.read_csv(r"data/twitter_training.csv")

# Validation Data
# validation_df = pd.read_csv(r"C:\Users\ebrady\Personal Projects\tweet-sentiment\data\twitter_validation.csv")
validation_df = pd.read_csv(r"data/twitter_validation.csv")

In [86]:
# Visualize the training data
training_df.sample(frac=1).head(10)

,ID,SOURCE,SENTIMENT,TWEET
32539,7582,LeagueOfLegends,Negative,League of Legends worse than heroin
34572,6739,Fortnite,Negative,When Fortnite was fun lol
10635,13029,Xbox(Xseries),Irrelevant,"Maybe because I've been involved in a few, too..."
35600,8113,Microsoft,Neutral,China rejects US tech company Microsoft's alle...
11909,8447,NBA2K,Irrelevant,"Shoutout to Women_shadows, bsschasker, and har..."
67656,7188,johnson&johnson,Negative,don’t care i’m not taking no vaccine at the en...
63393,7665,MaddenNFL,Irrelevant,BIG facts! –
72414,8806,Nvidia,Positive,Awesome - Nvidia RTX: Noise-destroying AI put ...
586,2501,Borderlands,Irrelevant,Since I would miss... borderlands tumblr so mu...
74621,9190,Nvidia,Positive,were


In [87]:
# Visualize split of source
training_df['SOURCE'].value_counts()

# Visualize split of sentiment
training_df['SENTIMENT'].value_counts()

Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: SENTIMENT, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder
import re

dfs = []
for df in [training_df, validation_df]:
    ### Remove Null Tweets
    df = df.dropna(subset=['TWEET'])
    cleaned_strings = []
    for string in df["TWEET"]:
        string = re.sub('[^A-Za-z0-9 ]+', '', string)
        cleaned_strings.append(string)
    df["TWEET"] = cleaned_strings
    
    ### Encode Source
    le = LabelEncoder()
    df['SOURCE'] = le.fit_transform(df['SOURCE'])

    
    ### Drop ID
    df = df.drop(columns={'ID'})
    
    ### Downsample
    from sklearn.utils import resample
    neg_sentiment = df[df["SENTIMENT"] == "Negative"]
    pos_sentiment = df[df["SENTIMENT"] == "Positive"]
    neu_sentiment = df[df["SENTIMENT"] == "Neutral"]
    neg_downsample = resample(neg_sentiment,
                         replace=True,
                         n_samples=len(neu_sentiment),
                         random_state=42)
    pos_downsample = resample(pos_sentiment,
                         replace=True,
                         n_samples=len(neu_sentiment),
                         random_state=42)

    ### Encode Sentiment
    df['SENTIMENT'] = le.fit_transform(df['SENTIMENT'])

    # Append dataframes
    dfs.append(df)

training_df, validation_df = dfs


In [ ]:
### Set character threshold


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Feature Vectorization
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vecs = vectorizer.fit_transform(training_df['TWEET'])
val_vecs = vectorizer.transform(validation_df['TWEET'])

In [ ]:
### Combine feature vectors with data

# import pandas as pd
# import scipy.sparse as sp

# # Convert the DataFrame to a NumPy array
# train_array = training_df['SOURCE'].values
# val_array = validation_df['SOURCE'].values

# # Convert the NumPy array to a SciPy sparse CSR matrix
# train_csr_matrix = scipy.sparse.csr_matrix(train_array)
# val_csr_matrix = scipy.sparse.csr_matrix(val_array)

# # Use the `scipy.sparse.csr_matrix.vstack()` function to concatenate the two matrices
# train_preproc = sp.vstack((train_csr_matrix, train_vecs))
# val_preproc = sp.vstack((val_csr_matrix, val_vecs))

In [ ]:
from sklearn import svm
from sklearn.metrics import classification_report

x_train = train_vecs
y_train = training_df['SENTIMENT']

x_val = val_vecs
y_val_true = validation_df['SENTIMENT']

# Perform classification with SVM, kernel=linear
linear_svm = svm.SVC(kernel='linear')
linear_svm.fit(x_train, y_train)
yval_pred = linear_svm.predict(x_val)

# Results
report = classification_report(validation_df['SENTIMENT'], prediction_linear, output_dict=True)
print('positive: ', report['pos'])
print('negative: ', report['neg'])


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_val_true, yval_pred)


In [ ]:
pickle.dump(linear_svm, open("linear_svm.pkl", "wb"))


In [ ]:

class TrainSentiment:
    """"""
    def __init__(self, twitter_train=settings.TWITTER_TRAINING, twitter_val=settings.TWITTER_VALIDATION):
        self.twitter_train = pd.read_csv(twitter_train)
        self.twitter_val   = pd.read_csv(twitter_val)
        self.dfs           = [self.twitter_train, self.twitter_val]
        self.train_preproc = None
        self.val_preproc   = None
        self.train_vecs    = None
        self.val_vecs      = None
        

    def preprocess(self):
        for df in self.dfs:

            ### Remove Null Tweets
            df = df.dropna(subset=['TWEET'])
            
            ### Encode Source
            le = LabelEncoder()
            df['SOURCE'] = le.fit_transform(df['SOURCE'])

            ### Encode Sentiment
            df['SENTIMENT'] = le.fit_transform(df['SENTIMENT'])

            ### Drop ID
            df = df.drop(columns={'ID'})
            
            # Append dataframes
            dfs.append(df)

        self.train_preproc, self.val_preproc = dfs

    def tf_idf(self):
        # Feature Vectorization
        vectorizer = TfidfVectorizer(min_df = 5,
                                    max_df = 0.8,
                                    sublinear_tf = True,
                                    use_idf = True)

        self.train_vecs = vectorizer.fit_transform(self.train_preproc['TWEET'])
        self.val_vecs = vectorizer.transform(self.val_preproc['TWEET'])

    def train_svm(self):
        # Perform classification with SVM, kernel=linear
        linear_svm = svm.SVC(kernel='linear')
        linear_svm.fit(self.train_vecs, self.train_preproc['SENTIMENT'])
        pickle.dump(linear_svm, open("linear_svm.pkl", "wb"))
        # yval_pred = linear_svm.predict(self.val_vecs)
